# ML workflow for  Chronic Kidney Disease

In [1]:
# Imports 
from pipeline_perso import *
from sklearn.pipeline import Pipeline

In [2]:
dtypes = {'pcv':'float64','wc':'float64','rc':'float64'}
data = pd.read_csv('kidney_disease.csv',dtype=dtypes,na_values='\t?')
data = data.dropna()

In [3]:
data

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
9,9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,...,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,ckd
11,11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,...,32.0,4500.0,3.8,yes,yes,no,poor,yes,no,ckd
14,14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,...,16.0,11000.0,2.6,yes,yes,yes,poor,yes,no,ckd
20,20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,...,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


In [4]:
data.reset_index(drop=True,inplace=True)
data.drop(['id'],axis=1,inplace=True)

In [5]:
data[data.columns[data.dtypes=='object']]

,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane,classification
0,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
1,abnormal,abnormal,present,notpresent,yes,yes,no,poor,no,yes,ckd
2,abnormal,abnormal,present,notpresent,yes,yes,no,poor,yes,no,ckd
3,normal,abnormal,present,present,yes,yes,yes,poor,yes,no,ckd
4,abnormal,abnormal,notpresent,notpresent,yes,yes,yes,poor,yes,yes,ckd
...,...,...,...,...,...,...,...,...,...,...,...
153,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
154,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
155,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd
156,normal,normal,notpresent,notpresent,no,no,no,good,no,no,notckd


In [6]:
TrainTest=TrainTestGenerator(1,test_size=0.3,data=data)
for train,test in TrainTest:
    X_train = np.array(data.drop(['classification'],axis=1))[train]
    X_test  = np.array(data.drop(['classification'],axis=1))[test]
    y_train = np.array(data.classification)[train]
    y_test  = np.array(data.classification)[test]

In [7]:
from sklearn.model_selection import KFold,GridSearchCV
kf = KFold(n_splits=4, random_state=None, shuffle=True)

In [8]:
from catboost import CatBoostClassifier
parameters = {'depth': [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],'iterations'    : [30, 50, 100]}

grid_perso = GridSearchHyperParamsCV(model=CatBoostClassifier(), parameters = parameters, cv_splitter = kf, n_jobs=-1 ,verbose=10)

grid = GridSearchCV(estimator=CatBoostClassifier(), param_grid = parameters, cv = kf, n_jobs=-1)

pipe_perso = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('catboost', grid_perso)])

pipe = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')), ('catboost', grid)])

pipe_perso.fit(X_train,y_train)

pipe.fit(X_train,y_train)

print(pipe_perso.score(X_test,y_test))
print(pipe_perso['catboost']._best_params)



print(pipe.score(X_test,y_test))
print(pipe['catboost'].best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  23 out of  27 | elapsed:    5.6s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    6.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1500s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  20 out of  27 | elapsed:    2.3s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    3.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1529s.) Setting batch_

0:	learn: 0.6758662	total: 53.6ms	remaining: 1.55s
1:	learn: 0.6599734	total: 56.4ms	remaining: 789ms
2:	learn: 0.6395456	total: 57.2ms	remaining: 515ms
3:	learn: 0.6242166	total: 58.7ms	remaining: 382ms
4:	learn: 0.6055429	total: 59.7ms	remaining: 299ms
5:	learn: 0.5902738	total: 60.7ms	remaining: 243ms
6:	learn: 0.5764389	total: 61.7ms	remaining: 203ms
7:	learn: 0.5626196	total: 62.7ms	remaining: 172ms
8:	learn: 0.5481796	total: 63.7ms	remaining: 149ms
9:	learn: 0.5359826	total: 64.7ms	remaining: 129ms
10:	learn: 0.5210264	total: 65.7ms	remaining: 113ms
11:	learn: 0.5070242	total: 66.7ms	remaining: 100ms
12:	learn: 0.4922983	total: 67.7ms	remaining: 88.5ms
13:	learn: 0.4821886	total: 68.7ms	remaining: 78.5ms
14:	learn: 0.4714849	total: 69.7ms	remaining: 69.7ms
15:	learn: 0.4590960	total: 70.7ms	remaining: 61.8ms
16:	learn: 0.4464068	total: 71.7ms	remaining: 54.8ms
17:	learn: 0.4352343	total: 72.7ms	remaining: 48.5ms
18:	learn: 0.4254588	total: 73.7ms	remaining: 42.7ms
19:	learn: 0.41

In [11]:
ct=CategoricalTransformer(strategy='ordinal_encoding')
fs=FeatureSelectorCorr(0.8)

a=ct.transform(X_train)
a=fs.transform(a)
print(a)
a.shape

[[37 60 1.02 ... 0 0 0]
 [41 70 1.02 ... 0 0 0]
 [42 70 1.025 ... 0 0 0]
 ...
 [57 60 1.02 ... 0 0 0]
 [50 90 1.01 ... 1 1 1]
 [48 70 1.005 ... 1 1 1]]


(111, 21)

In [12]:
from sklearn.svm import SVC
parameters =  {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

grid_perso = GridSearchHyperParamsCV(model=SVC(), parameters = parameters, cv_splitter = kf, n_jobs=-1 ,verbose=10)

grid = GridSearchCV(estimator=SVC(), param_grid = parameters, cv = kf, n_jobs=-1)

pipe_perso = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')),('fs',FeatureSelectorCorr(0.8)), ('svc', grid_perso)])

pipe = Pipeline([('cat_trans', CategoricalTransformer(strategy='ordinal_encoding')),('fs',FeatureSelectorCorr(0.8))
, ('svc', grid)])

pipe_perso.fit(X_train,y_train)

pipe.fit(X_train,y_train)

print(pipe_perso.score(X_test,y_test))
print(pipe_perso['svc']._best_params)



print(pipe.score(X_test,y_test))
print(pipe['svc'].best_params_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0068s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0252s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0068s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0113s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  27 out

ValueError: X.shape[1] = 20 should be equal to 21, the number of features at training time